
# Serving Extensions — vLLM & TensorRT-LLM (Cookbook)

이 노트북은 **vLLM**과 **TensorRT-LLM**을 활용한 서빙 확장 전반을 다룹니다.
- vLLM: PagedAttention, OpenAI-호환 서버, 동시성/처리량 설정
- TensorRT-LLM: 모델 변환 → 엔진 빌드 → 런타임 추론 (NVIDIA GPU 전용)

> 이 노트북은 환경에 따라 일부 셀은 **가이드/샘플 코드** 형태입니다.
> 실제 실행에는 해당 라이브러리/도구 설치와 GPU가 필요합니다.


## 1) vLLM — Quickstart & API Server

In [ ]:

# (가이드) 설치
# pip install vllm  # CUDA 환경 필요

print("vLLM quickstart — this cell contains guidance. Install vLLM in your environment to run.")


In [ ]:

# (샘플) 파이썬 내장 추론
# from vllm import LLM, SamplingParams
# llm = LLM(model="meta-llama/Llama-3-8B", tensor_parallel_size=1)  # 모델/권한 필요
# sampling_params = SamplingParams(temperature=0.7, top_p=0.9, max_tokens=64)
# outputs = llm.generate(["Hello, LLM!"], sampling_params)
# print(outputs[0].outputs[0].text)

print("See commented code for Python in-process vLLM usage.")



### 1.1 OpenAI 호환 서버 실행 (권장)
```bash
python -m vllm.entrypoints.openai.api_server \
  --model meta-llama/Llama-3-8B \
  --tensor-parallel-size 2 \
  --gpu-memory-utilization 0.90 \
  --max-num-batched-token 8192 \
  --port 8000
```
- **중요 옵션**
  - `--tensor-parallel-size`: 멀티 GPU 분산
  - `--gpu-memory-utilization`: VRAM 사용 상한
  - `--max-num-batched-token`: 동시성/처리량에 큰 영향



### 1.2 OpenAI 클라이언트로 호출 (Python)
```python
from openai import OpenAI
client = OpenAI(base_url="http://localhost:8000/v1", api_key="EMPTY")
resp = client.chat.completions.create(
    model="meta-llama/Llama-3-8B",
    messages=[{"role":"user","content":"Explain PagedAttention briefly."}],
    temperature=0.7, top_p=0.9
)
print(resp.choices[0].message.content)
```



### 1.3 성능 팁
- **`max_num_batched_token`**와 **`block_size`** 튜닝 → 처리량에 직접적 영향
- **KV 캐시 Offload**(CPU/디스크) 고려: 장문/동시성↑시 유용
- **CUDA 그래프**/Pinned memory 설정으로 지연시간 하향
- **prompt caching** 활성화로 반복 프롬프트 비용 절감


## 2) TensorRT-LLM — Build & Run


### 2.1 변환 및 엔진 빌드 개요
1. **HF 모델 → TensorRT-LLM 포맷 변환**
2. **엔진 빌드 (INT8/FP8/FP16 등 정밀도 선택)**
3. **런타임 추론 (`trtllm-run` 등)**

> 자세한 명령은 TensorRT-LLM 버전에 따라 다를 수 있습니다.


In [ ]:

# (가이드) 설치 및 버전 확인
# pip install tensorrt_llm
print("TensorRT-LLM steps are provided as guidance. Requires NVIDIA GPU/driver and TRT-LLM installed.")



### 2.2 변환 예시 (명령줄)
```bash
# 예시: Llama-3-8B를 TRT-LLM용으로 변환 (경로/옵션은 환경에 맞게 변경)
trtllm-build \
  --model_dir /models/Llama-3-8B \
  --dtype float16 \
  --tp_size 2 \
  --pp_size 1 \
  --output_dir /models/Llama-3-8B-trt
```
- `--dtype`: `float16`, `fp8`, `int8` 등 선택
- `--tp_size`/`--pp_size`: 텐서/파이프라인 병렬화



### 2.3 런타임 추론
```bash
trtllm-run \
  --engine_dir /models/Llama-3-8B-trt \
  --max_output_len 128 \
  --tokenizer_dir /models/Llama-3-8B
```
- 서버 모드(HTTP/gRPC) 실행 스크립트는 배포판/샘플에 포함되어 있는 경우가 많습니다.



### 2.4 성능 팁
- **정밀도**: `fp8`/`int8`로 내리면 처리량↑, 품질/안정성은 테스트 필요
- **프로파일링**: `nsys`, `ncu`로 커널/메모리 병목 확인
- **배치/패킹**: 긴 프롬프트 혼합 시 패킹으로 토큰 낭비 최소화
- **엔진 재사용**: 동일 모델/설정에서는 엔진 캐시 활용


## 3) 미니 벤치마크 스캐폴드 (공통)

In [ ]:

# 이 스캐폴드는 어떤 서빙 백엔드든 HTTP OpenAI 호환 엔드포인트가 있으면
# 간단히 처리량/지연시간을 측정할 수 있습니다.

import time, statistics, json, os
import threading
import queue
import requests

def load_prompts(n=50):
    base = "Write a short haiku about {}."
    topics = [f"topic-{i}" for i in range(n)]
    return [base.format(t) for t in topics]

def worker(url, model, prompts, results):
    for p in prompts:
        t0 = time.time()
        r = requests.post(
            f"{url}/chat/completions",
            headers={"Authorization": "Bearer EMPTY", "Content-Type":"application/json"},
            data=json.dumps({
                "model": model,
                "messages":[{"role":"user","content":p}],
                "temperature":0.7,
                "top_p":0.9,
                "max_tokens":64,
                "stream": False
            }),
            timeout=60
        )
        dt = time.time() - t0
        results.put(dt)

def run_benchmark(url="http://localhost:8000/v1", model="your-model", concurrency=8, total=64):
    prompts = load_prompts(total)
    per_worker = total // concurrency
    results = queue.Queue()

    threads = []
    for i in range(concurrency):
        subset = prompts[i*per_worker:(i+1)*per_worker]
        th = threading.Thread(target=worker, args=(url, model, subset, results))
        th.start()
        threads.append(th)

    for th in threads: th.join()

    times = []
    while not results.empty():
        times.append(results.get())

    if times:
        p50 = statistics.median(times)
        p90 = sorted(times)[int(0.9*len(times))-1]
        thpt = len(times) / sum(times)
        print(f"Requests: {len(times)}  |  p50: {p50:.3f}s  p90: {p90:.3f}s  |  Throughput: {thpt:.2f} req/s")
    else:
        print("No results collected. Check server URL/model and retry.")

print("Benchmark scaffold ready. Start your vLLM/TensorRT-LLM server and call run_benchmark().")



## 4) 마무리 팁
- **vLLM**: 동시성/배치 토큰 설정이 성능 핵심. KV 캐시 블록 사이즈와 메모리 사용률을 조절.
- **TensorRT-LLM**: 정밀도/병렬화 설정이 핵심. 변환/엔진 빌드는 느릴 수 있으나, 한 번 빌드하면 매우 빠름.
- 공통: **프롬프트 캐시**와 **장문 프롬프트 패킹**으로 토큰 낭비를 줄이세요.
